In [1]:
import os, json
from datetime import datetime, timedelta, timezone
import pandas as pd

In [2]:
import plotly.express as px
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# read files

In [3]:
dirname = "C:/Users/david/singularity/midfreq/scrappers/defillama_history/data/DynLst"
pools = []
for filename in os.listdir(dirname):
    fullname = os.path.join(os.sep, dirname, filename)
    data = pd.read_csv(fullname, parse_dates=['date']).set_index('date')['haircut_apy']
    pools.append(data.rename(filename.replace('.csv','')))
history = pd.concat(pools, axis=1)

# compute historical pool rank

In [24]:
max_rank = 3
end = datetime.now().replace(tzinfo=timezone.utc)
start = end - timedelta(days = 900)
df = history.fillna(method='ffill')
df = df[(df.index >= start)&(df.index <= end)]
best_only = df[df.rank(axis=1, ascending=False)<max_rank].dropna(how='all', axis=1)
best_only.iplot()

In [25]:
metadata = pd.read_csv("C:/Users/david/signal/scrappers/defillama_history/data/DynLst_pool_metadata.csv")
pool_map = metadata[['pool','project','symbol']].set_index('pool').T.to_dict()
pool_map = {key: '{}_{}'.format(x['project'], x['symbol']) for key, x in pool_map.items()}
index_map = {f'weight_{i}': f'w_{value}' for i, value in enumerate(pool_map.values())}
index_map |= {f'yield_{i}': f'y_{value}' for i, value in enumerate(pool_map.values())} 
pool_map

{'747c1d2a-c668-4682-b9f9-296708a3dd90': 'lido_STETH',
 'd4b3c522-6127-4b89-bedf-83641cdcd2eb': 'rocket-pool_RETH',
 'e6435aae-cbe9-4d26-ab2c-a4d533db9972': 'aave-v3_WSTETH',
 '77020688-e1f9-443c-9388-e51ace15cc32': 'frax-ether_SFRXETH',
 'e880e828-ca59-4ec6-8d4f-27182a4dc23d': 'aave-v3_WETH',
 '6d342d6a-9796-443e-8152-b94e8b6021fc': 'convex-finance_ETH-FRXETH',
 '5ce23e7e-3800-4c9c-ad30-6db3db0515a1': 'convex-finance_ETH-STETH',
 '42c02674-f7a2-4cb0-be3d-ade268838770': 'balancer-v2_RETH-WETH',
 'e378a7c2-6285-4993-9397-87ac9c8adc15': 'convex-finance_ETH-STETH',
 'f1a22245-a1f1-4389-8ff2-dde4914b272d': 'aave-v3_RETH',
 '4cc5df76-f81d-49fe-9e1e-4caa6a8dad0b': 'balancer-v2_WSTETH-WETH',
 'fc7701fc-e290-4606-a8be-9e4ba6c5f91a': 'convex-finance_ETH-RETH',
 'c9873dab-0979-478c-b48c-3c3a0c935449': 'convex-finance_STETH-FRXETH'}

In [26]:
with open(os.path.join(os.sep, dirname, f'EverTop{max_rank}.json'), 'w') as fp:
    json.dump(list(best_only.columns), fp=fp)

In [27]:
for it in best_only.columns:
    descr = pool_map[it]
    print(f'- \"{it}\" # {descr}')

- "42c02674-f7a2-4cb0-be3d-ade268838770" # balancer-v2_RETH-WETH
- "4cc5df76-f81d-49fe-9e1e-4caa6a8dad0b" # balancer-v2_WSTETH-WETH
- "5ce23e7e-3800-4c9c-ad30-6db3db0515a1" # convex-finance_ETH-STETH
- "747c1d2a-c668-4682-b9f9-296708a3dd90" # lido_STETH
- "77020688-e1f9-443c-9388-e51ace15cc32" # frax-ether_SFRXETH
- "c9873dab-0979-478c-b48c-3c3a0c935449" # convex-finance_STETH-FRXETH
- "d4b3c522-6127-4b89-bedf-83641cdcd2eb" # rocket-pool_RETH
- "e378a7c2-6285-4993-9397-87ac9c8adc15" # convex-finance_ETH-STETH
- "e6435aae-cbe9-4d26-ab2c-a4d533db9972" # aave-v3_WSTETH
- "f1a22245-a1f1-4389-8ff2-dde4914b272d" # aave-v3_RETH
- "fc7701fc-e290-4606-a8be-9e4ba6c5f91a" # convex-finance_ETH-RETH
